## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy
import sys
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Tiksi,71.69,128.87,18.46,96,100,18.01,RU,2020-10-18 17:32:19,light snow
1,1,Ushuaia,-54.80,-68.30,46.40,71,40,18.34,AR,2020-10-18 17:27:42,scattered clouds
2,2,Camalu,30.85,-116.07,84.00,19,22,1.01,MX,2020-10-18 17:32:20,few clouds
3,3,Roebourne,-20.78,117.13,66.99,72,0,12.97,AU,2020-10-18 17:32:20,clear sky
4,4,Springfield,37.22,-93.30,57.00,87,90,6.93,US,2020-10-18 17:28:58,mist


In [6]:
city_data_df["Max Temp"]=city_data_df["Max Temp"].astype("float64")

In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
mintemp=input("enter min temp")
maxtemp=input("enter max temp")

enter min temp75
enter max temp90


In [8]:
mintemp=float(mintemp)
type(mintemp)

float

In [9]:
maxtemp=float(maxtemp)
type(maxtemp)

float

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_data_df=city_data_df.loc[(city_data_df["Max Temp"] > mintemp) & (city_data_df["Max Temp"] < maxtemp)]
vacation_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
2,2,Camalu,30.85,-116.07,84.00,19,22,1.01,MX,2020-10-18 17:32:20,few clouds
8,8,Alofi,-19.06,-169.92,77.00,94,89,3.36,NU,2020-10-18 17:32:22,overcast clouds
10,10,Vila Velha,-20.33,-40.29,78.01,69,75,8.05,BR,2020-10-18 17:28:08,broken clouds
11,11,Butaritari,3.07,172.79,81.57,73,63,18.10,KI,2020-10-18 17:32:22,broken clouds
19,19,Bayir,37.27,28.22,75.20,73,75,9.17,TR,2020-10-18 17:32:25,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
700,700,Keti Bandar,24.14,67.45,85.42,72,67,5.57,PK,2020-10-18 17:36:39,broken clouds
703,703,Umm Lajj,25.02,37.27,84.04,57,0,8.50,SA,2020-10-18 17:36:39,clear sky
704,704,Kieta,-6.22,155.63,78.94,76,87,1.21,PG,2020-10-18 17:36:40,overcast clouds
705,705,Naifaru,5.44,73.37,82.69,76,74,11.05,MV,2020-10-18 17:36:40,broken clouds


In [11]:
# 4a. Determine if there are any empty rows.
vacation_data_df.count()

Unnamed: 0     225
City           225
Lat            225
Lng            225
Max Temp       225
Humidity       225
Cloudiness     225
Wind Speed     225
Country        222
Date           225
Description    225
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_data_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Camalu,MX,84.00,few clouds,30.85,-116.07,
8,Alofi,NU,77.00,overcast clouds,-19.06,-169.92,
10,Vila Velha,BR,78.01,broken clouds,-20.33,-40.29,
11,Butaritari,KI,81.57,broken clouds,3.07,172.79,
19,Bayir,TR,75.20,broken clouds,37.27,28.22,
21,Axim,GH,77.79,broken clouds,4.87,-2.24,
22,Cockburn Town,TC,87.80,scattered clouds,21.46,-71.14,
30,East Hemet,US,81.00,clear sky,33.74,-116.94,
31,Pombos,BR,84.20,scattered clouds,-8.14,-35.40,
49,Sola,VU,79.54,light rain,-13.88,167.55,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{Lat},{Lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [15]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Camalu,MX,84.00,few clouds,30.85,-116.07,La Cueva del Pirata
8,Alofi,NU,77.00,overcast clouds,-19.06,-169.92,Taloa Heights
10,Vila Velha,BR,78.01,broken clouds,-20.33,-40.29,Hotel Vitória Palace
11,Butaritari,KI,81.57,broken clouds,3.07,172.79,Isles Sunset Lodge
19,Bayir,TR,75.20,broken clouds,37.27,28.22,Buban Otel
...,...,...,...,...,...,...,...
700,Keti Bandar,PK,85.42,broken clouds,24.14,67.45,Haji laloo memon house
703,Umm Lajj,SA,84.04,clear sky,25.02,37.27,Moon light Furnished Units
704,Kieta,PG,78.94,overcast clouds,-6.22,155.63,
705,Naifaru,MV,82.69,broken clouds,5.44,73.37,Naifaru Residence Guest House


In [60]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df["Hotel Name"].replace("", "NaN")
hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Camalu,MX,84.00,few clouds,30.85,-116.07,La Cueva del Pirata
8,Alofi,NU,77.00,overcast clouds,-19.06,-169.92,Taloa Heights
10,Vila Velha,BR,78.01,broken clouds,-20.33,-40.29,Hotel Vitória Palace
11,Butaritari,KI,81.57,broken clouds,3.07,172.79,Isles Sunset Lodge
19,Bayir,TR,75.20,broken clouds,37.27,28.22,Buban Otel
...,...,...,...,...,...,...,...
697,Miranda,CO,88.00,scattered clouds,3.25,-76.23,Parqueadero toño lasso
700,Keti Bandar,PK,85.42,broken clouds,24.14,67.45,Haji laloo memon house
703,Umm Lajj,SA,84.04,clear sky,25.02,37.27,Moon light Furnished Units
705,Naifaru,MV,82.69,broken clouds,5.44,73.37,Naifaru Residence Guest House


In [61]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'clean_hotel_df' is not defined

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Maximum Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [90]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))